In [1]:
'''
    first lets import the neccesary labraries
    we import re which stand for regular expressions because we what to use it to remove the currency symbol 
    on the price 
'''
from bs4 import BeautifulSoup as bs4
import requests
import pandas as pd
import re

'''
    next we initiale the list of columns we what to fetch
'''

pages = []
prices = []
stars = []
titles = []
stock_availibility =[]
urlss = []

'''
    this variable will the number of pages we want to fetch information from
    this variable can be made dynamic where as a user can be prompt to enter the number of pages he wants.

'''
no_pages =5

'''
    this loop will iterate to the number of pages we want to fetch data from
    and append each url to the list of pages
    http://books.toscrape.com/catalogue/page-2.html
    looking at the url above we can see that before the .html has a figure 2
    show it's important that we are able to identify the pattern of the url
'''
for i in range (1, no_pages + 1):
    url = ('http://books.toscrape.com/catalogue/page-{}.html').format(i)
    pages.append(url)


'''
    now we have the number pages we want, we are going to iterate through each page and use the beautifulsoup library 
    to easy iterate through all the html tags 
'''
for item in pages:
    page = requests.get(item)
    soup = bs4(page.text, 'html.parser')
    
    
    '''
        each of this loops is going append the information collected to the list of columns we declared above
        the first two for loops are straight forward so there is nothing much to explain

    '''
    for i in soup.findAll('h3'):
        ttl = i.getText()
        titles.append(ttl)
    
    for n in soup.findAll('p', class_='instock availability'):
        stk = n.getText().strip()
        stock_availibility.append(stk)

    '''
        for the firt loop below, we can see it has another loop within it. this because looking at the inspected
        elements of the website, the rating is embeded int he string of the class definition see the sample below
        (<p class="star-rating One">) for we need to apply some trick to get only the value we want.
        class ['star-rating', 'Three']
        class ['star-rating', 'One']
        class ['star-rating', 'One']
    '''
    for s in soup.findAll('p', class_='star-rating'):
        for k,v in s.attrs.items():
            star = v[1]
            stars.append(star)
    '''
        in this loop too we can see why we imported the regular expression library.
        this is to remove the currency symbols for the price tag
    '''
    for j in soup.findAll('p',class_='price_color'):
        price = j.getText()
        trim = re.compile(r'[^\d.,]+')
        price = trim.sub('',price)
        prices.append(price)
    '''
        also like the above we need to apply some paitonic stuff so that we can get the image thumbnail url
    '''
    divs = soup.findAll('div', class_='image_container')
    for thumbs in divs:
        tags = thumbs.find('img', class_='thumbnail')
        urls = 'http://books.toscrape.com/'+str(tags['src'])
        newurls = urls.replace("../","")
        urlss.append(newurls)

In [2]:
'''
    Finally we save all the list of data as a dictionary because is easier to convert it to a data frame when the data 
    is in dictionary fomart.also we can save our data in csv, json or even insert it to our database
'''
dic = {'TITLE': titles, 'PRICE': prices, 'RATING': stars, 'STOCK': stock_availibility, 'URLs': urlss}
df = pd.DataFrame(data = dic)
df.head()

,TITLE,PRICE,RATING,STOCK,URLs
0,A Light in the ...,51.77,Three,In stock,http://books.toscrape.com/media/cache/2c/da/2c...
1,Tipping the Velvet,53.74,One,In stock,http://books.toscrape.com/media/cache/26/0c/26...
2,Soumission,50.10,One,In stock,http://books.toscrape.com/media/cache/3e/ef/3e...
3,Sharp Objects,47.82,Four,In stock,http://books.toscrape.com/media/cache/32/51/32...
4,Sapiens: A Brief History ...,54.23,Five,In stock,http://books.toscrape.com/media/cache/be/a5/be...
